In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm 
from PIL import Image
from io import BytesIO
import re

In [2]:
test_data = pd.read_csv('/share/jproject/fg538/r-006-gpu-4/data/test.csv')

In [3]:
test_data.head()

,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...
2,0001bbb682d45002,https://lh3.googleusercontent.com/-kloLenz1xZk...
3,0002362830cfe3a3,https://lh3.googleusercontent.com/-N6z79jNZYTg...
4,000270c9100de789,https://lh3.googleusercontent.com/-keriHaVOq1U...


In [4]:
TARGET_SIZE = 299

def overwrite_urls(df):
    def reso_overwrite(url_tail, reso=TARGET_SIZE):
        pattern = 's[0-9]+'
        search_result = re.match(pattern, url_tail)
        if search_result is None:
            return url_tail
        else:
            return 's{}'.format(reso)

    def join_url(parsed_url, s_reso):
        parsed_url[-2] = s_reso
        return '/'.join(parsed_url)

    parsed_url = df.url.apply(lambda x: x.split('/'))
    train_url_tail = parsed_url.apply(lambda x: x[-2])
    resos = train_url_tail.apply(lambda x: reso_overwrite(x, reso=TARGET_SIZE))

    overwritten_df = pd.concat([parsed_url, resos], axis=1)
    overwritten_df.columns = ['url', 's_reso']
    df['url'] = overwritten_df.apply(lambda x: join_url(x['url'], x['s_reso']), axis=1)
    return df

In [5]:
test_data_sub2 = overwrite_urls(test_data)

In [6]:
test_data_sub2.head()

,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...
2,0001bbb682d45002,https://lh3.googleusercontent.com/-kloLenz1xZk...
3,0002362830cfe3a3,https://lh3.googleusercontent.com/-N6z79jNZYTg...
4,000270c9100de789,https://lh3.googleusercontent.com/-keriHaVOq1U...


In [7]:
len(test_data_sub2)

117703

In [8]:
for i in tqdm(range(58000)):
    URL = test_data_sub2['url'][i]
    #class_id = str(train_data_sub2['landmark_id'][i])
    image_name = test_data_sub2['id'][i]
    filename = '/share/jproject/fg538/r-006-gpu-4/data/test_data/' + image_name + '.jpg'

    # skip download if it already exists
    if os.path.exists(filename):
        continue

    try:
        with urllib.request.urlopen(URL) as url:
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            image = url.read()
    except:
        #tqdm.write('Could not download image: {}'.format(image_name)) # print() interferes with progress bar
        pass
        
    try:
        pil_image = Image.open(BytesIO(image))
    except:
        #tqdm.write('Failed to parse image: {}'.format(image_name))
        pass
        
    try:
        pil_image_rgb = pil_image.convert('RGB')
    except:
        #tqdm.write('Failed to convert image {} to RGB'.format(image_name))
        pass

    try:
        pil_image_resize = pil_image_rgb.resize((299, 299))
    except:
        pass
    
    try:
        pil_image_resize.save(filename, format='JPEG', quality=90)
    except:
        #tqdm.write('Failed to save image: {}'.format(filename))
        pass

  3%|▎         | 1644/58000 [05:19<3:02:25,  5.15it/s]/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
100%|██████████| 58000/58000 [4:24:45<00:00,  3.65it/s]  


In [ ]:
5